In [1]:
!pip install keras
!pip install tensorflow
!pip install pandas
!pip install matplotlib
!pip install sklearn

  Obtaining dependency information for keras from https://files.pythonhosted.org/packages/fe/58/34d4d8f1aa11120c2d36d7ad27d0526164b1a8ae45990a2fede31d0e59bf/keras-2.14.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
    --------------------------------------- 0.0/1.7 MB 991.0 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.7 MB 1.4 MB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.7 MB 1.7 MB/s eta 0:00:01
   ------ --------------------------------- 0.3/1.7 MB 1.8 MB/s eta 0:00:01
   --------- ------------------------------ 0.4/1.7 MB 1.7 MB/s eta 0:00:01
   ---------- ----------------------------- 0.5/1.7 MB 1.7 MB/s eta 0:00:01
   ------------- -------------------------- 0.6/1.7 MB 1.8 MB/s eta 0:00:01
   ---------------- ----------------------- 0.7/1.7 MB 1.8 MB/s eta 0:00:01
   ----------------- ---------------------- 0.8/1.7 MB 1.9 MB/s eta 0:00:01
   -------------------- ------------------- 0.9/1

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sklearn: filename=sklearn-0.0.post9-py3-none-any.whl size=2970 sha256=fa7163dce2e7cc6d64fd38c216349978a80045e71d68fbdafb3c3b124bd6a2ed
  Stored in directory: c:\users\hitesh patil\appdata\local\pip\cache\wheels\ef\63\d1\f1671e1e93b7ef4d35df483f9b2485e6dd21941da9a92296fb
Successfully built sklearn
